In [39]:
from deep_translator import GoogleTranslator
from transformers import pipeline
import pandas as pd
import numpy as np
import gc

sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/opt/anaconda3/envs/sentiment_analysis_2/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
# enter the path of the csv table down below
table_path = "/Users/gamzezorlubas/Desktop/sentiment_analysis/reviews.csv"
data_reviews = pd.read_csv(table_path)

In [40]:
data_reviews["translated_text"] = ""
def translate_reviews(reviews, batch_size = 20):
    translated_reviews = []
    for i in range(0, len(reviews), batch_size):
        batch_reviews = reviews[i:i + batch_size]
        batch_text = "|||".join(batch_reviews)
        # Save batch_text to a file
        with open("batch.txt", "w", encoding="utf-8") as file:
            file.write(batch_text)

        translated_batch = GoogleTranslator(source='german', target='english').translate_file("batch.txt")
        translated_batch = translated_batch.split("|||")
        if len(translated_batch) != batch_size:
            translated_batch = GoogleTranslator(source='german', target='english').translate_batch(batch_reviews)
            print(f"fallback solution for batch: {int((i/batch_size) + 1)}")
        translated_reviews.extend(translated_batch) 
        print(f"processing batch: {int((i/batch_size) + 1)}")
        gc.collect()
    return translated_reviews

reviews_to_translate = data_reviews["text"].tolist()
translated_reviews = translate_reviews(reviews_to_translate, batch_size = 20)
data_reviews["translated_text"] = translated_reviews

fallback solution for batch: 1
processing batch: 1
processing batch: 2
processing batch: 3
processing batch: 4
processing batch: 5
fallback solution for batch: 6
processing batch: 6
fallback solution for batch: 7
processing batch: 7
fallback solution for batch: 8
processing batch: 8
processing batch: 9
processing batch: 10
processing batch: 11
processing batch: 12
processing batch: 13
processing batch: 14
processing batch: 15
processing batch: 16
processing batch: 17
processing batch: 18
processing batch: 19
fallback solution for batch: 20
processing batch: 20
processing batch: 21
processing batch: 22
processing batch: 23
processing batch: 24
processing batch: 25
processing batch: 26
processing batch: 27
fallback solution for batch: 28
processing batch: 28
processing batch: 29
processing batch: 30
processing batch: 31
fallback solution for batch: 32
processing batch: 32
processing batch: 33
processing batch: 34
processing batch: 35
processing batch: 36
processing batch: 37
processing b

In [43]:
data_reviews["sentiment"] = ""
num_reviews = len(data_reviews["text"])
for i in range(num_reviews): 
    designer_review = data_reviews["translated_text"][i]
    # print(data_reviews["text"][i])
    # print(data_reviews["translated_text"][i])
    data_reviews.loc[i, "sentiment"] = sentiment_pipeline(designer_review)[0]["label"]
    print(f"finding sentiment of review {i}")

finding sentiment of review 0
finding sentiment of review 1
finding sentiment of review 2
finding sentiment of review 3
finding sentiment of review 4
finding sentiment of review 5
finding sentiment of review 6
finding sentiment of review 7
finding sentiment of review 8
finding sentiment of review 9
finding sentiment of review 10
finding sentiment of review 11
finding sentiment of review 12
finding sentiment of review 13
finding sentiment of review 14
finding sentiment of review 15
finding sentiment of review 16
finding sentiment of review 17
finding sentiment of review 18
finding sentiment of review 19
finding sentiment of review 20
finding sentiment of review 21
finding sentiment of review 22
finding sentiment of review 23
finding sentiment of review 24
finding sentiment of review 25
finding sentiment of review 26
finding sentiment of review 27
finding sentiment of review 28
finding sentiment of review 29
finding sentiment of review 30
finding sentiment of review 31
finding sentiment 

In [45]:
data_reviews.to_csv("reviews_with_sentiment.csv")

A filtering might be based on the checking of three conditions:
  * rating<=4
  * occurence of at least one of the keywords
  * sentiment result as NEGATIVE